In [1]:
import meshcat
import meshcat.geometry as geom
import meshcat.transformations as tf
import numpy as np
import time
import jax.numpy as jnp
import jax
from jaxlie import SE2, SE3
import matplotlib.pyplot as plt
import math

In [2]:
########################################## Parameters ##########################################
_m1 = 0.1 #mass
_m2 = 0.1
_m3 = 0.1

_l1 = 0.3 #lenght
_l2 = 0.3
_l3 = 0.1

I1 = 1/3 * _m1 * _l1**2 #Inertia
I2 = 1/3 * _m2 * _l2**2

dt = 1e-5 # 1e-4 #time step
_g = 9.81 #gravity
steps = 10000 # iterations

### link 1 ###
q1 = 0.35 #rads
q_dot1 = 0 #rads/s
T1 = 0

### link 2 ###
q2 = 0
q_dot2 = 0
T2 = 0

In [3]:
def mass_matrix(q1, q2):
    m11 = I1 + I2 + _m2 * (_l1 ** 2) + 2 * _m2 * _l1 * _l2 * math.cos(q2)
    m12 = I2 + _m2 * _l1 * _l2 * math.cos(q2)
    m21 = I2 + _m2 * _l1 * _l2 * math.cos(q2)
    m22 = I2
    return np.array([ [m11, m12], [m21, m22] ])
    
def coriolis(q2, q_dot1, q_dot2):
    c11 = -2 * _m2 * _l1 * _l2 * math.sin(q2) * q_dot2
    c12 = -_m2 * _l1 * _l2 * math.sin(q2) * q_dot2
    c21 = _m2 * _l1 * _l2 * math.sin(q2) * q_dot1
    c22 = 0.0

    return np.array([ [c11, c12], [c21, c22] ])
    
def gravy(q1, q2):
    _lc1 = _l1 / 2
    _lc2 = _l2 / 2
    g1 = -_m1 * _g * _lc1 * math.sin(q1) - _m2 * _g * (_l1 * math.sin(q1) + _lc2 * math.sin(q1 + q2))
    g2 = -_m2 * _g * _lc2 * math.sin(q1 + q2)
    return np.array([[g1],[g2]])


def euler(dt, q1, q2, q_dot1, q_dot2, q_ddot1, q_ddot2):
    q_dot1 = q_dot1 + (q_ddot1 * dt)
    q1 = q1 + (q_dot1 * dt)
    
    q_dot2 = q_dot2 + (q_ddot2 * dt)
    q2 = q2 + (q_dot2 * dt)
    return q1.item(), q2.item(), q_dot1.item(), q_dot2.item()

In [4]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
### pivot 1 ### 
pivot_1 = vis["base_pivot"].set_object(geom.Sphere(0.01))
### link 1 ###
link = vis["pendulum_link"]
link.set_object(geom.Box([0.01, 0.01, -_l1]))

### pivot 2 ###
vis["base_pivot_2"].set_object(geom.Sphere(0.01))

### link 2 ###
link2 = vis["pendulum_link_2"]
link2.set_object(geom.Box([0.01, 0.01, -_l2]))

for t in range(steps):
    mass = mass_matrix(q1, q2)
    c_matrix = coriolis(q2, q_dot1, q_dot2)
    gravity = gravy(q1, q2)
    B = np.array([ [0], [1] ]) #for T2 only?
    torque = np.array([[T2]])
    
    q_dot = np.array([ [q_dot1], [q_dot2] ])

    # c_q_dot = B @ torque - c_matrix @ q_dot + gravity

    # q_ddot = np.linalg.solve(mass, c_q_dot).flatten()

    q_ddot = np.linalg.inv(mass) @ (B @ torque - c_matrix @ q_dot + gravity)

    q_ddot1, q_ddot2 = q_ddot[0], q_ddot[1]
    q1, q2, q_dot1, q_dot2 = euler(dt, q1, q2, q_dot1, q_dot2, q_ddot1, q_ddot2)
    print(q1, q2, q_dot1, q_dot2)

    ### link 1 ### 
    r_1 = tf.rotation_matrix(q1, [0, 1, 0])
    t_1 = tf.translation_matrix([0.0, 0.0, -(_l1 / 2)])
    
    vis["pendulum_link"].set_transform(np.dot(r_1, t_1))
    
    ### pivot 2 ###
    pt_2 = tf.translation_matrix([0.0, 0.0, - _l1])
    vis["base_pivot_2"].set_transform(np.dot(r_1, tf.translation_matrix([0.0, 0.0, - _l1])))

    ### link 2 ###
    t_2 = np.dot(r_1, tf.translation_matrix([0.0, 0.0, -_l1]))  # relative to origin
    r_2 = tf.rotation_matrix(q2, [0.0, 1.0, 0.0])
    t2 = tf.translation_matrix([0.0, 0.0, -_l2/2])

    vis["pendulum_link_2"].set_transform(np.dot(t_2, np.dot(r_2, t2)))
    
    time.sleep(dt)

0.35 -1.6819137455689883e-09 -1.0239361308424449e-20 -0.0001681913745568988
0.35 -5.045741247556439e-09 4.649774575235408e-13 -0.00033638275019874506
0.35 -1.0091482527661303e-08 1.8599099153378097e-12 -0.0005045741280104863
0.35000000000000003 -1.6819137618432003e-08 4.649774859709426e-12 -0.0006727655090770701
0.35000000000000014 -2.5228706563266444e-08 9.29954981638598e-12 -0.000840956894483444
0.3500000000000003 -3.5320189416412e-08 1.627421237697682e-11 -0.001009148285314556
0.3500000000000006 -4.709358624296554e-08 2.603874013699142e-11 -0.0011773396826553536
0.350000000000001 -6.054889711887339e-08 3.9058110692852e-11 -0.001345531087590785
0.35000000000000153 -7.568612213093137e-08 5.579730177421902e-11 -0.0015137225012057983
0.3500000000000023 -9.250526137678479e-08 7.672129118032691e-11 -0.001681913924585342
0.3500000000000033 -1.1100631496492844e-07 1.0229505670410544e-10 -0.0018501053588143648
0.35000000000000464 -1.311892830147066e-07 1.329835762734023e-10 -0.00201829680497